In [48]:
import numpy as np
with open("Sistemas de prueba\IEEE34.atp", "r+") as f:
    lines = f.readlines()
lines_copy = lines.copy()

# Extraer Lineas de las cargas en archivo
three_phase_loads = []
single_phase_loads = []
switches = []
for idx, line in enumerate(lines_copy):
    
    if "C Load" in line:
        if "3f" in line:
            three_phase_loads.append(lines_copy[idx:idx+4])
            
        else:
            single_phase_loads.append(lines_copy[idx:idx+2])
    if "/BRANCH" in line:
        branch_idx = idx
    if "/SWITCH" in line:
        switch_idx = idx
ti = 0.01
tf = 0.06


In [15]:
a = ["C Load 836 3f                                                                  0\n",
"  X0016AX0016B                     33067.2         16533.6                     0\n",
"  X0016BX0016C                91177.941176    54706.764706                     0\n",
"  X0016CX0016A                23167.633452    12135.427046                     0\n",
"C Load 816 3f                                                                  0\n",
"  X0002AX0002B                       1.E30           1.E30                     0\n",
"  X0002BX0002C                213796.55172     85518.62069                     0\n",
"  X0002CX0002A                       1.E30           1.E30                     0\n",
"  X0002AX0002B                       1.E30           1.E30                     0\n",
"  X0002BX0002C                213796.55172     85518.62069                     0\n",
"  X0002CX0002A                       1.E30           1.E30                     0\n",
"  X0002AX0002B                       1.E30           1.E30                     0\n",
"  X0002BX0002C                213796.55172     85518.62069                     0\n",
"  X0002CX0002A                       1.E30           1.E30                     0\n"]

indices = []
for idx, line in reversed(list(enumerate(a))):
    if "1.E30" in line:
        del a[idx]
for line in a:
    print(line, end='')
# for idx in indices:
#     a.remove(idx)

C Load 836 3f                                                                  0
  X0016AX0016B                     33067.2         16533.6                     0
  X0016BX0016C                91177.941176    54706.764706                     0
  X0016CX0016A                23167.633452    12135.427046                     0
C Load 816 3f                                                                  0
  X0002BX0002C                213796.55172     85518.62069                     0
  X0002BX0002C                213796.55172     85518.62069                     0
  X0002BX0002C                213796.55172     85518.62069                     0


In [49]:
three_phase_ti = np.random.uniform(ti, tf, (len(three_phase_loads),3))
three_phase_ti = np.around(three_phase_ti, 5)
single_phase_ti = np.random.uniform(ti, tf, len(single_phase_loads))
single_phase_ti = np.around(single_phase_ti, 5)

In [50]:
# Iterar por cada Carga
from utils.components import single_phase_switch, three_phase_switch

tff = str(tf).rjust(10)
for load_idx, loads in enumerate(three_phase_loads):
    switch = three_phase_switch.copy()
    switch_node = []
    for idx, load in enumerate(loads):
        if idx == 0:
            load_node = f"L{load_idx}"
            loads[idx] = f"{load.strip()}-Load Change".ljust(80) + "\n"
            switch_node.append(f"{load.strip()} - switch".ljust(80) + "\n")
            continue

        node_1 = load[2:8]
        node_2 = load[8:14].strip()

        phase_1 = node_1.strip()[-1]
        load_node_phase = f"{load_node}{phase_1}".center(6)

        phase_2 = ""
        load_node_phase_2 = "".center(6)

        if node_2:
            phase_2 = node_2[-1]
            load_node_phase_2 = f"{load_node}{phase_2}".center(6)

        switch_ln = switch[idx]
        ti = str(three_phase_ti[load_idx, idx - 1]).rjust(10)

        # Create new Lines
        switch[
            idx
        ] = f"{switch_ln[:2]}{node_1}{load_node_phase}{ti}{tff}{switch_ln[34:]}"
        loads[idx] = f"{load[:2]}{load_node_phase}{load_node_phase_2}{load[14:]}"
        switch_node.append(switch[idx])
    switches.append(switch_node)

for load_idx, loads in enumerate(single_phase_loads):
    switch = single_phase_switch.copy()
    switch_node = []
    for idx, load in enumerate(loads):
        if idx == 0:
            load_node = f"L{load_idx}"
            loads[idx] = f"{load.strip()}-Load Change mono".ljust(80) + "\n"
            switch_node.append(f"{load.strip()} - switch mono".ljust(80) + "\n")
            continue

        node_1 = load[2:8]
        node_2 = load[8:14].strip()

        phase_1 = node_1.strip()[-1]
        load_node_phase = f"{load_node}{phase_1}".center(6)

        phase_2 = ""
        load_node_phase_2 = "".center(6)

        if node_2:
            phase_2 = node_2[-1]
            load_node_phase_2 = f"{load_node}{phase_2}".center(6)

        switch_ln = switch[idx]
        ti = str(single_phase_ti[load_idx]).rjust(10)

        # Create new Lines
        switch[
            idx
        ] = f"{switch_ln[:2]}{node_1}{load_node_phase}{ti}{tff}{switch_ln[34:]}"
        loads[idx] = f"{load[:2]}{load_node_phase}{load_node_phase_2}{load[14:]}"
        switch_node.append(switch[idx])
    switches.append(switch_node)

        


In [60]:
a = 154893.56806508
aa = f"{a}"[:12].center(10)
print(aa)
154893.56806

154893.56806


154893.56806

In [51]:
# for loads in three_phase_loads:
#     print('\n')
#     for load in loads:
#         print(load, end='')

# for switch in switches:
#     print('\n')
#     for line in switch:
#         print(line, end='')
with open("Sistemas de prueba\IEEE34.atp", "r+") as f:
    lines = f.readlines()
lines_copy = lines.copy()
for switch in switches:
    lines_copy[switch_idx + 2 : switch_idx + 2] = switch
for load in three_phase_loads:
    lines_copy[branch_idx + 3 : branch_idx + 3] = load
for load in single_phase_loads:
    lines_copy[branch_idx + 3 : branch_idx + 3] = load
with open("Sistemas de prueba\prueba.atp", "w+") as f:
    f.writelines(lines_copy)


In [35]:
import plotly.express as px
import pandas as pd
from utils_tesis import signalload

signals = signalload.CSV_pandas()
print(signals.labels_list)
signal, t, _ = signals.load_data('I: X0071C-R3C')
df = pd.DataFrame(signal, t)
px.line(df)

C:/Users/aherrada/Documents/Git/SEM_Repo/3. Archivos CSV/L043.65_048.58.csv
Se seleccionó L043.65_048.58
['V: R1A', 'V: R1B', 'V: R1C', 'V: R2A', 'V: R2B', 'V: R2C', 'V: R3A', 'V: R3B', 'V: R3C', 'I: X0038A-X0043A', 'I: X0038B-X0043B', 'I: X0038C-X0043C', 'I: X0048A-X0053A', 'I: X0048B-X0053B', 'I: X0048C-X0053C', 'I: X0060A-X0065A', 'I: X0060B-X0065B', 'I: X0060C-X0065C', 'I: X0066A-X0067A', 'I: X0066B-X0067B', 'I: X0066C-X0067C', 'I: X0023A-R1A', 'I: X0023B-R1B', 'I: X0023C-R1C', 'I: X0004A-R2A', 'I: X0004B-R2B', 'I: X0004C-R2C', 'I: X0071A-R3A', 'I: X0071B-R3B', 'I: X0071C-R3C', 'MACH 1: ID', 'MACH 1: IQ', 'MACH 1: I0', 'MACH 1: IF', 'MACH 1: IKD', 'MACH 1: IG', 'MACH 1: IKQ', 'MACH 1: IA', 'MACH 1: IB', 'MACH 1: IC', 'MACH 1: EFD', 'MACH 1: MFORCE', 'MACH 1: MANG', 'MACH 1: TQ GEN', 'MACH 1: ANG 1', 'MACH 1: VEL 1', 'MODELS: IQMIN', 'MODELS: IDMAX', 'MODELS: IQMIN', 'MODELS: IDMAX', 'MODELS: IQMIN', 'MODELS: IDMAX', 'MODELS: IQMIN', 'MODELS: IDMAX']


In [ ]:
164.76 => 0.04451 t
51.57 => 0.04609 t  (844 3f fase A)